In [1]:
#192.168.43.256
#all imports
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
def grader_tf_version():
    assert((tf.__version__)>'2')
    return True
grader_tf_version()

True

In [4]:
#Read the dataset - Amazon fine food reviews
review = pd.read_csv(r"Reviews.csv")
#check the info of the dataset
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [5]:
reviews=pd.DataFrame()
reviews["score"]=review['Score']
reviews["text"]=review['Text']
reviews.shape

(568454, 2)

In [6]:
reviews.isnull().sum()
#no null values

score    0
text     0
dtype: int64

In [7]:
new_score=[]
for score in reviews["score"]:
    if score > 3:
        new_score.append(1)
    elif score <= 2:
        new_score.append(0)
    else:
        continue

In [8]:
len(new_score)

525814

In [9]:
reviews=reviews[reviews.score!=3]
reviews.shape

(525814, 2)

In [10]:
reviews["Score"]=new_score
reviews.head(5)

,score,text,Score
0,5,I have bought several of the Vitality canned d...,1
1,1,Product arrived labeled as Jumbo Salted Peanut...,0
2,4,This is a confection that has been around a fe...,1
3,2,If you are looking for the secret ingredient i...,0
4,5,Great taffy at a great price. There was a wid...,1


In [11]:
reviews = reviews.drop('score', 1)

In [12]:
def grader_reviews():
    temp_shape = (reviews.shape == (525814, 2)) and (reviews.Score.value_counts()[1]==443777)
    assert(temp_shape == True)
    return True
grader_reviews()

True

In [13]:
def get_wordlen(x):
    return len(x.split())
reviews['len'] = reviews.text.apply(get_wordlen)
reviews = reviews[reviews.len<50]
reviews = reviews.sample(n=100000, random_state=30)

In [14]:
reviews.shape

(100000, 3)

In [15]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(clean, '', text).lower()

In [16]:
from tqdm import tqdm
Text=[]
for text in tqdm(reviews['text']):
    Text.append(remove_html_tags(text))

    
    
    

100%|██████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 155632.57it/s]


In [17]:
len(Text)

100000

In [18]:
import re
def decontracted(phrase):
    """function for processing text data and removing unwanted characters"""
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"\\n", " ", phrase)
    phrase = re.sub(r"\\r", " ", phrase)
    phrase = re.sub(r"\\", " ", phrase)
    phrase = re.sub('[^A-Z0-9a-z]+', " ", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [19]:
# Import stopwords with nltk.
text=[]
from nltk.corpus import stopwords
stop = stopwords.words('english')
for sentence in tqdm(Text):
    sent = decontracted(sentence)
    sent = ' '.join(e for e in sent.split() if e not in stopwords.words())
    text.append(sent.lower().strip())


100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [6:10:46<00:00,  4.50it/s]


In [26]:
reviews = reviews.drop('Text', 1)

In [27]:
reviews["Text"]=text
y=reviews['Score'].values
X=reviews.drop('Score', 1)


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                                    test_size=0.20,
                                                    random_state=33,
                                                    stratify=y)

In [29]:
## Loading the Pretrained Model from tensorflow HUB
tf.keras.backend.clear_session()
# maximum length of a seq in the data we have, for now i am making it as 75. You can change this
max_seq_length = 55
#BERT takes 3 inputs
#this is input words. Sequence of words represented as integers
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")

#mask vector if you are padding anything
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")

#segment vectors. If you are giving only one sentence for the classification, total seg vector is 0. 
#If you are giving two sentenced with [sep] token separated, first seq segment vectors are zeros and 
#second seq segment vector are 1's
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")

#bert layer 
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable=False)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
bert_model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=pooled_output)


In [30]:
bert_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 55)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 55)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 55)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [31]:
bert_model.output

<tf.Tensor 'keras_layer/Identity:0' shape=(None, 768) dtype=float32>

In [32]:
#getting Vocab file
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [33]:
import tokenization
tokenizer = tokenization.FullTokenizer(vocab_file,do_lower_case)

In [34]:
#it has to give no error 
def grader_tokenize(tokenizer):
    out = False
    try:
        out=('[CLS]' in tokenizer.vocab) and ('[SEP]' in tokenizer.vocab)
    except:
        out = False
    assert(out==True)
    return out
grader_tokenize(tokenizer)

True

In [82]:
def convert_text(sentence,max_seq,tokenizer):
    max_seq-=2
    all_tokens=[]
    mask=[]
    segment=[]
    for i in range(sentence.shape[0]):
        token=tokenizer.tokenize(sentence[i])
        if len(token)> max_seq:
            token=token[:max_seq]
        tokens= ['[CLS]',*token,'[SEP]']  
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+token +["[SEP]"])+[0] * (max_seq - len(token))
        mask_array=([1]*len(tokens)+ [0]*(max_seq +2 -len(tokens)))
        seg_array=([0]*(max_seq+2))
        all_tokens.append(one_token)
        mask.append(mask_array)
        segment.append(seg_array)
    return np.array(all_tokens),np.array(mask),np.array(segment)

In [83]:
train_text=X_train['Text'].values
test_text=X_test["Text"].values

X_train_tokens,X_train_mask,X_train_segment=convert_text(train_text,max_seq_length,tokenizer)
X_test_tokens,X_test_mask,X_test_segment=convert_text(test_text,max_seq_length,tokenizer)



In [85]:
def grader_alltokens_train():
    out = False
    
    if type(X_train_tokens) == np.ndarray:
        
        temp_shapes = (X_train_tokens.shape[1]==max_seq_length) and (X_train_mask.shape[1]==max_seq_length) and \
        (X_train_segment.shape[1]==max_seq_length)
        
        segment_temp = not np.any(X_train_segment)
        
        mask_temp = np.sum(X_train_mask==0) == np.sum(X_train_tokens==0)
        
        no_cls = np.sum(X_train_tokens==tokenizer.vocab['[CLS]'])==X_train_tokens.shape[0]
        
        no_sep = np.sum(X_train_tokens==tokenizer.vocab['[SEP]'])==X_train_tokens.shape[0]
        
        out = temp_shapes and segment_temp and mask_temp and no_cls and no_sep
      
    else:
        print('Type of all above token arrays should be numpy array not list')
        out = False
    assert(out==True)
    return out

grader_alltokens_train()

True

In [86]:
import pickle
##save all your results to disk so that, no need to run all again. 
pickle.dump((X_train, X_train_tokens, X_train_mask, X_train_segment, y_train),open('train_data.pkl','wb'))
pickle.dump((X_test, X_test_tokens, X_test_mask, X_test_segment, y_test),open('test_data.pkl','wb'))

In [87]:
#you can load from disk
X_train, X_train_tokens, X_train_mask, X_train_segment, y_train = pickle.load(open("train_data.pkl", 'rb')) 
X_test, X_test_tokens, X_test_mask, X_test_segment, y_test = pickle.load(open("test_data.pkl", 'rb')) 

In [89]:
X_train_tokens.shape

(80000, 55)

In [90]:
def grader_alltokens_test():
    out = False
    if type(X_test_tokens) == np.ndarray:
        
        temp_shapes = (X_test_tokens.shape[1]==max_seq_length) and (X_test_mask.shape[1]==max_seq_length) and \
        (X_test_segment.shape[1]==max_seq_length)
        
        segment_temp = not np.any(X_test_segment)
        
        mask_temp = np.sum(X_test_mask==0) == np.sum(X_test_tokens==0)
        
        no_cls = np.sum(X_test_tokens==tokenizer.vocab['[CLS]'])==X_test_tokens.shape[0]
        
        no_sep = np.sum(X_test_tokens==tokenizer.vocab['[SEP]'])==X_test_tokens.shape[0]
        
        out = temp_shapes and segment_temp and mask_temp and no_cls and no_sep
      
    else:
        print('Type of all above token arrays should be numpy array not list')
        out = False
    assert(out==True)
    return out
grader_alltokens_test()

True

In [91]:
bert_model.input

[<tf.Tensor 'input_word_ids:0' shape=(None, 55) dtype=int32>,
 <tf.Tensor 'input_mask:0' shape=(None, 55) dtype=int32>,
 <tf.Tensor 'segment_ids:0' shape=(None, 55) dtype=int32>]

In [92]:
bert_model.output

<tf.Tensor 'keras_layer/Identity:0' shape=(None, 768) dtype=float32>

In [93]:
X_train_pooled_output=bert_model.predict([X_train_tokens,X_train_mask,X_train_segment])

In [94]:
X_test_pooled_output=bert_model.predict([X_test_tokens,X_test_mask,X_test_segment])

In [95]:
pickle.dump((X_train_pooled_output, X_test_pooled_output),open('final_output.pkl','wb'))

In [96]:
X_train_pooled_output, X_test_pooled_output= pickle.load(open('final_output.pkl', 'rb'))

In [97]:
#now we have X_train_pooled_output, y_train
#X_test_pooled_ouput, y_test

#please use this grader to evaluate
def greader_output():
    assert(X_train_pooled_output.shape[1]==768)
    assert(len(y_train)==len(X_train_pooled_output))
    assert(X_test_pooled_output.shape[1]==768)
    assert(len(y_test)==len(X_test_pooled_output))
    assert(len(y_train.shape)==1)
    assert(len(X_train_pooled_output.shape)==2)
    assert(len(y_test.shape)==1)
    assert(len(X_test_pooled_output.shape)==2)
    return True
greader_output()

True